In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.ensemble import VotingClassifier
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model, load_model

In [2]:
root_path = '.'

## Google Colab

In [3]:
try:
    from google.colab import drive
    drive.mount('/content/drive')

    root_path = '/content/drive/MyDrive/projects/facial_feeling_detection'
except:
    pass

In [4]:
try:
    from tensorflow.python.client import device_lib

    device_lib.list_local_devices()
except:
    pass

## Constants

In [5]:
SEED = 42
LABELS = {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}
TRAIN_PATH = root_path + '/data/train/'
TEST_PATH = root_path + '/data/test/'
MODELS_PATH = root_path + '/models/'
MODEL_PATH = MODELS_PATH + 'model_val_acc_0.6104_230225215228.h5'
SUBMISSIONS_PATH = root_path + '/submissions/'

## Data Loading

In [6]:
data = np.load(root_path + '/data/unbalanced_data.npz')
X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train']

with open(root_path + '/data/test_ids.pkl', 'rb') as f:
    test_ids = pickle.load(f)

submission_ids = pd.read_csv(root_path + '/data/test_set.csv')

## Model Set Up

In [7]:
model = load_model(MODEL_PATH)

vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))
vgg_model = Model(inputs=vgg16.input, outputs=vgg16.output)

for layer in vgg_model.layers:
    layer.trainable = False

In [8]:
X_vgg = np.stack((X_train,) * 3, axis=-1)

## Ensemble

In [9]:
ensemble = VotingClassifier(
    estimators=[('vgg', vgg_model), ('model', model)],
    voting='hard'
)

In [10]:
y_pred_vgg = vgg_model.predict(X_vgg)
y_pred_model = model.predict(X_train)

ensemble.fit([y_pred_vgg, y_pred_model], y_train)

901/901 [==============================] - 17s 18ms/step


ValueError: The estimator Functional should be a classifier.

## Prediction/Submission

In [ ]:
y_pred_ensemble = ensemble.predict([vgg_model.predict(X_test), model.predict(X_test)])

221/221 [==============================] - 4s 19ms/step


NotFittedError: This VotingClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.